# WhisperTS joining JSON to SRT

In [13]:
import re,codecs,json,unidecode
import pandas as pd

In [14]:
repJson="/Users/gilles/Github/AMBK-SportsCom/Whisper/Extrait/Transcripts/"

In [15]:
json1=repJson+"extrait-00.json"
with open(json1) as inFile:
    whisper1=json.load(inFile)

### Gestion des timestamps

In [16]:
def second2timestamp(seconds):
    sHours=int(seconds/3600)
    sMinutes=int((seconds-sHours*3600)/60)
    sSeconds=int((seconds-sHours*3600-sMinutes*60))
    sMilliseconds=int((seconds%1)*1000)
    return "%02d:%02d:%02d,%03d"%(sHours,sMinutes,sSeconds,sMilliseconds)

def timestamp2second(timestamp):
    m=re.match("(\d\d):(\d\d):(\d\d),(\d\d\d)",timestamp)
    if m:
        seconds=(int(m.group(1))*60+int(m.group(2)))*60+int(m.group(3))+float(m.group(4))/1000
        return seconds
    else:
        print("problème de timestamp")
        return

### Gestion des structures JSON de Whisper

In [17]:
def getWhisper(whisper,elements="segments"):
    lElements=[]
    if elements=="segments":
        for element in whisper[elements]:
            lElements.append((second2timestamp(element["start"]),
                              second2timestamp(element["end"]),
                              element["text"],
                              element["confidence"]))
    elif elements=="words":
        for segment in whisper["segments"]:
            for element in segment[elements]:
                lElements.append((second2timestamp(element["start"]),
                                  second2timestamp(element["end"]),
                                  element["text"],
                                  element["confidence"]))        
    return lElements

## Collage des segments

In [18]:
def removePunct(texte):
    result=texte
    punct = list('''!()-[]{};:'"\,<>./?@#$%^&*_~''')
    for p in punct:
        result=result.replace(p,"")
    return result
    
def egaleTextes(text1,text2):
    t1=unidecode.unidecode(removePunct(text1))
    t2=unidecode.unidecode(removePunct(text2))
    lMax=min(len(t1),len(t2))
    lMax=min(20,lMax)
    return t1[:lMax].lower()==t2[:lMax].lower()
    
def pasteSegment(srt,whisper2):
    for i in range(1,len(whisper2["segments"])):
        id1=None
        for n,(_,_,texte,_) in enumerate(srt):
            texte2=whisper2["segments"][i]["text"]
            if egaleTextes(texte,texte2):
                id1=n
                print(texte)
                print(texte2)
                break
        if id1:
            break
        else:
            print("problème de jointure",i, whisper2["segments"][i]["text"])
    if id1:
        jointTime=timestamp2second(srt[id1][0])
        newSrt=srt[:id1]
    
        srt2=getWhisper(whisper2)[i:]
        for (tDeb,tFin,texte,confidence) in srt2:
            nDeb=second2timestamp(timestamp2second(tDeb)+jointTime)
            nFin=second2timestamp(timestamp2second(tFin)+jointTime)
            newSrt.append((nDeb,nFin,texte,confidence))
        return newSrt
    else:
        return None

In [19]:
timestamp2second("00:01:00,000")

60.0

In [20]:
srt=[]
decalageInitial=0
for (tDeb,tFin,texte,confidence) in getWhisper(whisper1):
    nDeb=second2timestamp(timestamp2second(tDeb)+decalageInitial)
    nFin=second2timestamp(timestamp2second(tFin)+decalageInitial)
    srt.append((nDeb,nFin,texte,confidence))

In [21]:
for i in range(1,44):
    print(i)
    json2=repJson+"extrait-%02d.json"%i
    with open(json2) as inFile:
        whisper2=json.load(inFile)
    newSrt=pasteSegment(srt,whisper2)
    if not newSrt:
        break
    else:
        srt=newSrt
# print(srt)

1
problème de jointure 1  Lancé pour le capitaine de l'équipe de France, Raphaël Ibañez, qui a donc le choix de la combinaison.
 Un ballon bien pris, à quoi que relâché, et ça profite à Pichot.
 Un ballon bien pris, à quoi que relâché, et ça profite à Pichot. Un petit coup de pied par-dessus,
2
 Les Argentins qui ont bien démarré en chipant un premier ballon à l'alignement français et en s'emparant proprement du deuxième.
 Les Argentins qui ont bien démarré en chipant un premier ballon à l'alignement français
3
 Il faut vite aller recouper le terrain pour se rassurer.
 Il faut vite aller recouper le terrain pour se rassurer parce qu'en tout cas, c'est le début de partie, il est argentin, vous le voyez.
4
 Quand on est au sol, on est un joueur modeste.
 Quand on est au sol, on est un joueur mort, Thierry, on ne doit plus rien faire, si ce n'est que s'écarter le plus vite possible de la zone de jeu.
5
 Et c'est vrai, vous le verrez, que Mignoni, il a un dynamisme viral, alors que Pichott

In [22]:
# srt

In [23]:
srtLines=[]
for i,(start,end,mot,confidence) in enumerate(srt):
    n=i+1
    srtLines.append(str(n))
    srtLines.append(start+" --> "+end)
    srtLines.append(mot)
    srtLines.append("")


In [24]:
with codecs.open(json1.replace(".json","-segments.srt"),"w",encoding="utf8") as outFile:
    for srtLine in srtLines:
        outFile.write(srtLine+"\n")